# Join the Dots - Part B.
For a overview guide to what is happening here, please visit: https://www.youtube.com/watch?v=yaeFJiJiuhg

In [ ]:
import tifffile
import math
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
from ijroi.ij_roi import Roi
from ijroi.ijpython_decoder import decode_ij_roi
import cv2
import time
import multiprocessing
from find_maxima import find_maxima
from PIL import Image
import numpy as np
from scipy import ndimage
import time
import cv2
#Now we find the nearest dot to each non-dot.
def find_local_maxima_np(img_data):
    #This is the numpy/scipy version of the above function (find local maxima).
    #Its a bit faster, and more compact code.
    
    #Filter data with maximum filter to find maximum filter response in each neighbourhood
    max_out = ndimage.filters.maximum_filter(img_data,size=7)
    #Find local maxima.
    local_max = np.zeros((img_data.shape))
    imgc_data = np.copy(img_data)
    imgc_data[imgc_data<100] = 0
    local_max[max_out == imgc_data] = 1
    local_max[imgc_data == np.min(imgc_data)] = 0
    return local_max.astype(np.bool)
def detect_the_dots(img):
    ntol = 50
    img_data = np.array(img).astype(np.float64)

   
    #img_data[img_data<128] = 0

    #Should your image be an RGB image.
    if img_data.shape.__len__() >2:
        img_data = (np.sum(img_data,2)/3.0)

    if np.max(img_data) >255 or np.min(img_data)<0:
        print ('warning: your image should be scaled between 0 and 255 (8-bit).')

    #Finds the local maxima using maximum filter.
    local_max = find_local_maxima_np(img_data)
    
    
    y,x,out = find_maxima(img_data,local_max.astype(np.uint8),ntol)
    imgout = (out>=12)
    contours,hierarchy = cv2.findContours(np.array(imgout).astype(np.uint8), mode=cv2.RETR_EXTERNAL,method=1)
    dots_coords = []
    for cnt in contours[1:]:
        perimeter = float(cv2.arcLength(cnt,True))
        if perimeter >0:
            x,y,w,h = cv2.boundingRect(cnt)

            area = float(cv2.contourArea(cnt))
            circ = 4.*np.pi*(area/(perimeter**2))


            s = float(h)/float(w)
            if s > 1.0:
                pass
            else:
                s = float(h)/float(w)
            
            if circ >=0.7 and s <1.2:
                dots_coords.append([x,y,w,h])
                
    return dots_coords

def calculate_polar_features(v,num_of_con):
    dist_dot_ndot = compare_dist(v[:,0],v[:,1],v[:,0],v[:,1])
    ##Create angle based feature for each point.
    std = np.argsort(dist_dot_ndot,0)
    dot_corres = []
    for i in range(0,dist_dot_ndot.shape[0]):
        idxt = std[0:num_of_con,i] #with 0:number of neareast neighbours.
        dot_corres.append(idxt)
    dot_feat = []
    
    for  xy,arrXY in zip(v,dot_corres):
            
            origin = [xy[0],xy[1]]
            degrees =[]
            pts = []
            for tid in range(0,arrXY.shape[0]):
                pt = v[arrXY[tid]]
                pts.append(pt)
                rad,dst = clockwiseangle_and_distance(pt,origin)
                degrees.append(np.rad2deg(rad))

            ids =  np.argsort(np.array(degrees))
            degrees  = np.array(degrees)[ids]
            pts = np.array(pts)[ids]
            diff_arr = list(np.diff(degrees))
            diff_arr.append(360-np.sum(diff_arr))
            carr = (np.array(diff_arr)//20*20).astype(np.int)
            
            if carr.shape[0] == num_of_con:
                out_feat = np.zeros((carr.shape))
                min_id = np.argmin(carr)
                #print("min",min_id)
                out_feat[0:num_of_con-min_id] = carr[min_id:]
                if min_id !=0:
                    out_feat[num_of_con-min_id:] = carr[:min_id]
                dot_feat.append(out_feat)
    return dot_feat
def clockwiseangle_and_distance(point,origin):
    refvec = [0, 1]
    # Vector between point and the origin: v = p - o
    vector = [point[0]-origin[0], point[1]-origin[1]]
    # Length of vector: ||v||
    lenvector = math.hypot(vector[0], vector[1])
    # If length is zero there is no angle
    if lenvector == 0:
        return -math.pi, 0
    # Normalize vector: v/||v||
    normalized = [vector[0]/lenvector, vector[1]/lenvector]
    dotprod  = normalized[0]*refvec[0] + normalized[1]*refvec[1]     # x1*x2 + y1*y2
    diffprod = refvec[1]*normalized[0] - refvec[0]*normalized[1]     # x1*y2 - y1*x2
    angle = math.atan2(diffprod, dotprod)
    # Negative angles represent counter-clockwise angles so we need to subtract them 
    # from 2*pi (360 degrees)
    if angle < 0:
        return 2*math.pi+angle, lenvector
    # I return first the angle because that's the primary sorting criterium
    # but if two vectors have the same angle then the shorter distance should come first.
    return angle, lenvector
def compare_dist(dist_x0,dist_y0,dist_x1,dist_y1):

    dist_x0 = dist_x0.astype(np.float32)
    dist_y0 = dist_y0.astype(np.float32)
    dist_x1 = dist_x1.astype(np.float32)
    dist_y1 = dist_y1.astype(np.float32)
   
    m_x0 = np.tile(dist_x0, (dist_x1.shape[0],1)).astype(np.float)
    m_x1 = np.tile(dist_x1, (dist_x0.shape[0],1)).astype(np.float)
    m_y0 = np.tile(dist_y0, (dist_y1.shape[0],1)).astype(np.float)
    m_y1 = np.tile(dist_y1, (dist_y0.shape[0],1)).astype(np.float)
    m_x2 = m_x0-m_x1.T; m_y2 = m_y0-m_y1.T

    dist = np.sqrt(m_x2**2+m_y2**2)
    mlim = np.min(dist.shape)
    dist[np.arange(0,mlim),np.arange(0,mlim)] = 9999

    return dist
def decode_roi(filepath):
    overlay_arr = []
    names = []
    output = []

    if filepath.split(".")[-1]== "tif" or filepath.split(".")[-1]== "tiff" :
        tfile = tifffile.TiffFile(filepath)
        img = tfile.asarray()
        img_shape = tfile.asarray().shape[0:2]

        if 'Overlays' in tfile.imagej_metadata:
            overlays = tfile.imagej_metadata['Overlays']
            if overlays.__class__.__name__ == 'list':
                #Multiple overlays and so iterate.
                for overlay in overlays:

                    overlay_arr.append(decode_ij_roi(overlay,img_shape))
        #plt.imshow(img)
        for i in range(0,overlay_arr.__len__()):

            if overlay_arr[i] != False:
                x0 = int(overlay_arr[i].x)
                y0 = int(overlay_arr[i].y)
                wid = int(overlay_arr[i].width)
                hei = int(overlay_arr[i].height)
                name = overlay_arr[i].name
                #plt.plot([x0,x0+wid,x0+wid,x0,x0],[y0,y0,y0+hei,y0+hei,y0])
                output.append([x0,y0,wid,hei])
                names.append(name.replace('\x00',''))
        return output, names,img

def return_vid_arr(path):
    cap = cv2.VideoCapture(path)
    frame_arr = []
    while cap.isOpened():
        ret, frame = cap.read()
        # rotate cw
        out=cv2.transpose(frame)
        out=cv2.flip(out,flipCode=1)
        # if frame is read correctly ret is True
        if ret == False:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        frame_arr.append(out)
    
    cap.release()
    cv2.destroyAllWindows()
    return frame_arr


In [ ]:
template_path = "/Users/dwaithe/Documents/youtube_videos/computerVisionProjects/join_the_dots/images/scanned_with_lines/"
output_path = "/Users/dwaithe/Documents/youtube_videos/computerVisionProjects/join_the_dots/images/videos_final/"
input_path = "/Users/dwaithe/Documents/youtube_videos/computerVisionProjects/join_the_dots/images/videos_raw/"

import sys
#egg.
filename ='IMG_0068.png'
frame_arr = return_vid_arr(input_path+'IMG_6096.MOV')
out_video_name ="egg_homog.avi"
#affine = False
#eggb.
#filename ='IMG_0068.png'
#frame_arr = return_vid_arr(input_path+'IMG_6095.MOV')
#out_video_name ="egg_homogb.avi"
#violin.
#filename ='IMG_0058.png'
#frame_arr = return_vid_arr(input_path+'IMG_6088.MOV')
#out_video_name ="violin_homog.avi"
#
affine = False

#two chicks.
#filename ='IMG_0065.png'
#frame_arr = return_vid_arr(input_path+'IMG_5996.MOV')
#out_video_name ="two_chicks_homog.avi"
#
affine = False
#piano
#filename = "IMG_0067.png"
#frame_arr = return_vid_arr(input_path+'IMG_6094.MOV')
#out_video_name ="piano_homog.avi"
#piano
#filename = "IMG_0067.png"
#frame_arr = return_vid_arr(input_path+'IMG_6070.MOV') 
#out_video_name ="piano_homog2.avi"

#bird
#filename = "IMG_0062.png"
#frame_arr = return_vid_arr(input_path+'IMG_6104.MOV') 
#out_video_name ="bird_flight_homo2.avi"

#girl with recorder
#filename = "IMG_0045.png"
#frame_arr = return_vid_arr(input_path+'IMG_6108.MOV') 
#out_video_name ="girl_with_recorder.avi"

### Main script to compare two images for correspondence

In [ ]:


img = cv2.imread(template_path+filename)
fline = open(template_path+filename.split(".")[-2]+'.txt')
lines = fline.readlines()
lines_to_draw0 =[]
lines_to_draw1 =[]
rigid_mask = 0
font= cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,500)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2
for line in lines:
    prts = line.replace("(","").replace(")","").strip("\n").split(",")
    
    lines_to_draw0.append([int(prts[0]),int(prts[1])])
    lines_to_draw1.append([int(prts[2]),int(prts[3])])
    

plotss = []
for reg in lines_to_draw0:
        x,y= reg
        plotss.append([x,y])
num_of_con=6

XY = np.array(plotss)
dot_feat = calculate_polar_features(XY,num_of_con)
count = 0
size = (frame_arr[0].shape[1],frame_arr[0].shape[0])
outpath_video = output_path+out_video_name
out = cv2.VideoWriter(outpath_video, cv2.VideoWriter_fourcc(*"MJPG"), 30, size)
print(outpath_video)
deviation_ave =[]
from multiprocessing import Process, Queue, Pool

for ct in range(0, frame_arr.__len__()):
    t0 = time.time()
    img1 = np.copy(frame_arr[ct])
    gray = 255-cv2.cvtColor(np.copy(img1), cv2.COLOR_BGR2GRAY).astype(np.float32)
    
    
    #gray1 = np.copy(gray[960:1920,0:540])
    #gray2 = np.copy(gray[0:960,540:1080])
    #gray3 = np.copy(gray[960:1920,540:1080])
    
    
    output0 = detect_the_dots(gray)
    #output1 = detect_the_dots(gray1)
    #output2 = detect_the_dots(gray2)
    #output3 = detect_the_dots(gray3)
    t1 = time.time()
    
    
    plotsss = []
    for reg in output0:
            x,y,w,h = reg
            plotsss.append([x+w//2,y+h//2])
    
    
    XYs = np.array(plotsss)
    dot_feat1 =[[0,0],[0,0]]
    if XYs !=[]:
        dot_feat1 = calculate_polar_features(XYs,num_of_con) 
   
    
        if dot_feat1 != []:
            bf = cv2.BFMatcher(cv2.NORM_HAMMING2, crossCheck=False)
            matches = bf.match(np.uint8(dot_feat1),np.uint8(dot_feat))
            #to_keep = compare(dot_feat,dot_feat1,num_of_con-1)
            matches = sorted(matches, key = lambda x:x.distance)
            to_keep = []
            for m in matches:
                to_keep.append([m.trainIdx,m.queryIdx])
            

            src_pts =[]
            dst_pts = []
            for tok in to_keep[0:80]:
                xy0 = XY[tok[0],:]
                xy1 = XYs[tok[1],:]
                src_pts.append(xy0)
                dst_pts.append(xy1)


        if dst_pts !=[]:
            #transformation_rigid_matrix, rigid_mask = cv2.estimateAffinePartial2D(np.array(src_pts), np.array(dst_pts))
            M0, rigid_mask = cv2.findHomography(np.array(src_pts), np.array(dst_pts), cv2.RANSAC,5)
           
            if affine == True:
                affine0 = np.array(src_pts)#[np.where(rigid_mask)[0],:]
                affine1 = np.array(dst_pts)#[np.where(rigid_mask)[0],:]
                M0 = cv2.estimateAffinePartial2D(affine0,affine1)[0]
            if np.sum(rigid_mask) >5:
                M = M0

            if affine == True:
                dstd_pts = np.matmul(np.float32(XYs),M[:,:2].T) + M[:,2]
                ltt_pts0 = np.matmul(np.float32(lines_to_draw0),M[:,:2].T) + M[:,2]
                ltt_pts1 = np.matmul(np.float32(lines_to_draw1),M[:,:2].T) + M[:,2]
                ltt_pts0 = ltt_pts0.astype(np.int)
                ltt_pts1 = ltt_pts1.astype(np.int)

            else:
                dstd_pts = cv2.perspectiveTransform(np.float32(XYs).reshape(1,-1,2), M)[0]
                dstd_pts = dstd_pts.astype(np.int)
                ltt_pts0 = cv2.perspectiveTransform(np.float32(lines_to_draw0).reshape(1,-1,2), M)[0]
                ltt_pts1 = cv2.perspectiveTransform(np.float32(lines_to_draw1).reshape(1,-1,2), M)[0]


            


        

            dist_arr0 = compare_dist(ltt_pts0[:,0],ltt_pts0[:,1],XYs[:,0],XYs[:,1])
            idx = np.argmin(dist_arr0,0)
            dist_ = np.mean(dist_arr0[idx,:])
            tdx = np.where(dist_arr0[idx,:] <40)[0]

            if tdx.size >= 10:
                    M1, rigid_mask = cv2.findHomography(np.float32(ltt_pts0[tdx,:]),np.float32(XYs[idx,:])[tdx,:], cv2.RANSAC,5)
                    if np.sum(rigid_mask) >25:
                        ltt_pts0f = cv2.perspectiveTransform(np.float32(ltt_pts0).reshape(1,-1,2), M1)[0]
                        ltt_pts1f = cv2.perspectiveTransform(np.float32(ltt_pts1).reshape(1,-1,2), M1)[0]

                        deviation1 = np.median(np.abs(np.array(ltt_pts0f)-np.array(ltt_pts0)))
                        THR = 400
                        

                        
                        
                        deviation_mean = np.median(deviation_ave)
                        deviation_ave.append(deviation1)
                        #print('test',deviation_mean,deviation1,abs(deviation_mean-deviation1)/deviation_mean)
                        if 1.0 > abs(deviation_mean-deviation1)/deviation_mean or deviation_ave.__len__() == 1:
                            
                            #print('pass')
                            for lpts0,lpts1 in zip(ltt_pts0f,ltt_pts1f):

                                img1 = cv2.line(img1, (lpts0[0],lpts0[1]),(lpts1[0],lpts1[1]),(255,0,0),3)
                            oltt_pts0f = ltt_pts0f
                            oltt_pts1f = ltt_pts1f
                        else:
                            
                            for lpts0,lpts1 in zip(oltt_pts0f,oltt_pts1f):

                                img1 = cv2.line(img1, (lpts0[0],lpts0[1]),(lpts1[0],lpts1[1]),(255,0,0),3)
                            
                        #cv2.putText(img1,str(M0[0,:]), 
                                #(10,100), 
                                #font, 
                                #fontScale,
                                #fontColor,
                                #lineType)
                            #cv2.putText(img1,str(M0[1,:]), 
                                #(10,140), 
                                #font, 
                                #fontScale,
                                #fontColor,
                                #lineType)
                            cv2.putText(img1,str(dist_), 
                                (10,180), 
                                font, 
                                fontScale,
                                fontColor,
                                lineType)
    #cv2.imwrite(output_path + 'out'+str(ct)+".png",img1)
    out.write(img1)
    t2 = time.time() 
    count+=1
    print("time",t2-t1,t1-t0)
    #plt.imshow(img1)
    #plt.plot(dstd_pts[:,0],dstd_pts[:,1],'o')
    #plt.ylim(img1.shape[0],0)
    #plt.xlim(0,img1.shape[1])
out.release()

### Visualization tool for seeing correspondence.

In [ ]:
img1 = frame_arr[ct]
h = max(img.shape[0],img1.shape[0])
imgcon = np.zeros((h,img.shape[1]+img1.shape[1],3))
imgcon[:img.shape[0],:img.shape[1],:] = img
imgcon[:img1.shape[0],img.shape[1]:img.shape[1]+img1.shape[1],:] = img1
plt.figure(figsize=(16,16))
plt.imshow(imgcon.astype(np.int))
plt.plot(XY[:,0],XY[:,1],'ko')
plt.plot(img.shape[1]+XYs[:,0],XYs[:,1],'ko')
plt.plot(np.array(src_pts)[:,0],np.array(src_pts)[:,1],'o')
plt.plot(img.shape[1]+np.array(dst_pts)[:,0],np.array(dst_pts)[:,1],'o')
for src, dst in zip(src_pts,dst_pts):
    plt.plot([src[0],img.shape[1]+dst[0]],[src[1],dst[1]],'g-')
#for i in range(0, dstd_pts.shape[0]):
#    plt.plot(img.shape[1]+dstd_pts[i,0],dstd_pts[i,1],'go')
for lpts0,lpts1 in zip(ltt_pts0,ltt_pts1):
    plt.plot([img.shape[1]+lpts0[0],img.shape[1]+lpts1[0]],[lpts0[1],lpts1[1]],'b-')
